<a href="https://colab.research.google.com/github/carlosmding/DLFundamentals/blob/main/02--Aplicacion_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import os, sys
import cv2 as cv
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow as tf
import tensorflow.keras as k

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
rutaPath = '/content/drive/MyDrive/BASE_DATOS_SALIDA'
%cd '/content/drive/MyDrive'

/content/drive/MyDrive


Cargar Vector Imágenes y vector de Salidas

In [9]:
X = np.load(rutaPath + '/vectorImagenesCNN.npy')
print(X.shape)

y = np.load(rutaPath + '/vectorYCNN.npy')
print(y.shape)


(275, 128, 128, 3)
(275,)


Normalizar imágenes - Generar Array X

In [10]:
X_norm = X/np.linalg.norm(255.0)
X_norm = np.array(X_norm)

Salidad Categórica - Generar Array y

In [11]:
y=np.array(y)
y_categorical = to_categorical(y, num_classes=3)

## 0: Apple
## 1: Banana
## 2: Orange

Selección del Train/Test

In [12]:
# 70% Train y 30% Test
x_train, x_test, y_train, y_test = train_test_split(X_norm, y_categorical, test_size=0.3,random_state=109)
print(x_train.shape)
print(y_test.shape)

(192, 128, 128, 3)
(83, 3)


**Primer modelo construcción manual**


In [13]:
def get_conv_model(num_classes=3, img_size=128, use_maxpooling=True, compile=True):
    print ("using",num_classes,"classes")
    inputs = tf.keras.Input(shape=(img_size,img_size,3), name="input_1") #h,w,canal
    layers = tf.keras.layers.Conv2D(30,(3,3), activation="relu")(inputs)
    if use_maxpooling:
        layers = tf.keras.layers.MaxPool2D((2,2), name="pooling")(layers)

    layers = tf.keras.layers.Conv2D(30,(5,5), activation="relu")(layers)
    layers = tf.keras.layers.MaxPool2D((2,2), name="pooling2")(layers)  #0.84  (128,3,3)
    layers = tf.keras.layers.Flatten()(layers)
    layers = tf.keras.layers.Dense(60, activation=tf.nn.relu)(layers)  #128
    layers = tf.keras.layers.Dropout(0.2)(layers)
    predictions = tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax, name="output_1")(layers)
    model = tf.keras.Model(inputs = inputs, outputs=predictions)

    if compile:
        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    return model

In [14]:
modely = get_conv_model(use_maxpooling=True)
modely.summary()

using 3 classes
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 30)      840       
                                                                 
 pooling (MaxPooling2D)      (None, 63, 63, 30)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 59, 59, 30)        22530     
                                                                 
 pooling2 (MaxPooling2D)     (None, 29, 29, 30)        0         
                                                                 
 flatten (Flatten)           (None, 25230)             0         
                                                                 
 dense (Dense)               (None, 60)      

Entrenamiento

In [15]:
def train(model, batch_size, epochs, model_name=""):
    from time import time
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir="logs/"+model_name+"_"+"{}".format(time()))
    model.reset_states()
    model.fit(x_train,y_train, epochs=epochs, callbacks=[tensorboard],
              batch_size=batch_size,
              validation_data=(x_test, y_test))
    metrics = model.evaluate(x_test, y_test)

    return {k:v for k,v in zip (model.metrics_names, metrics)}

In [16]:
train(modely, batch_size=64, epochs=50, model_name="model_C")

Epoch 1/50
3/3 [==============================] - 7s 489ms/step - loss: 1.9631 - accuracy: 0.3177 - val_loss: 1.0921 - val_accuracy: 0.3012
Epoch 2/50
3/3 [==============================] - 0s 73ms/step - loss: 1.0760 - accuracy: 0.4167 - val_loss: 1.0934 - val_accuracy: 0.3012
Epoch 3/50
3/3 [==============================] - 0s 73ms/step - loss: 0.9719 - accuracy: 0.4792 - val_loss: 1.1031 - val_accuracy: 0.3012
Epoch 4/50
3/3 [==============================] - 0s 77ms/step - loss: 0.8813 - accuracy: 0.5521 - val_loss: 0.8348 - val_accuracy: 0.6024
Epoch 5/50
3/3 [==============================] - 0s 50ms/step - loss: 0.7117 - accuracy: 0.7240 - val_loss: 0.5325 - val_accuracy: 0.8795
Epoch 6/50
3/3 [==============================] - 0s 55ms/step - loss: 0.5753 - accuracy: 0.8125 - val_loss: 0.5551 - val_accuracy: 0.6747
Epoch 7/50
3/3 [==============================] - 0s 50ms/step - loss: 0.5206 - accuracy: 0.7552 - val_loss: 0.2956 - val_accuracy: 0.9277
Epoch 8/50
3/3 [==========

{'loss': 0.26047244668006897, 'accuracy': 0.9156626462936401}

Modelo ResNet152

In [17]:
res_model = k.applications.ResNet152(include_top=False, weights="imagenet",input_shape=(128,128,3))
res_model.summary()

234698864/234698864 [==============================] - 6s 0us/step
Model: "resnet152"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 134, 134, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 64, 64, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 64, 64, 64)           256       ['conv1_conv[0][0]']          
 on)                   

Modificaciones adicionales

In [18]:
model = k.models.Sequential()
model.add(k.layers.Lambda(lambda image: tf.image.resize(image, (128,128))))
model.add(res_model)
model.add(k.layers.Flatten())
model.add(k.layers.BatchNormalization())
model.add(k.layers.Dense(256, activation='relu'))
model.add(k.layers.Dropout(0.5))
model.add(k.layers.BatchNormalization())
model.add(k.layers.Dense(128, activation='relu'))
model.add(k.layers.Dropout(0.5))
model.add(k.layers.BatchNormalization())
model.add(k.layers.Dense(64, activation='relu'))
model.add(k.layers.Dropout(0.5))
model.add(k.layers.BatchNormalization())
model.add(k.layers.Dense(3, activation='softmax'))

In [19]:
file_path="/content/drive/MyDrive/Models_Resnet152/"
check_point = k.callbacks.ModelCheckpoint(filepath=file_path+"Model_Resnet152_corales.h5",monitor="val_accuracy",mode="max",save_best_only=True,)


In [20]:
model.compile(loss='categorical_crossentropy',optimizer=k.optimizers.RMSprop(learning_rate=2e-6),metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=16, epochs=1000, verbose=1,validation_data=(x_test, y_test),callbacks=[check_point])
model.summary()

Epoch 1/1000
12/12 [==============================] - ETA: 0s - loss: 1.6271 - accuracy: 0.3281

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12/12 [==============================] - 100s 2s/step - loss: 1.6271 - accuracy: 0.3281 - val_loss: 1.1069 - val_accuracy: 0.3253
Epoch 2/1000
12/12 [==============================] - 2s 190ms/step - loss: 1.5247 - accuracy: 0.3854 - val_loss: 1.1336 - val_accuracy: 0.3012
Epoch 3/1000
12/12 [==============================] - 2s 190ms/step - loss: 1.4753 - accuracy: 0.3750 - val_loss: 1.1603 - val_accuracy: 0.3012
Epoch 4/1000
12/12 [==============================] - 2s 194ms/step - loss: 1.4813 - accuracy: 0.3854 - val_loss: 1.2185 - val_accuracy: 0.3133
Epoch 5/1000
12/12 [==============================] - 3s 219ms/step - loss: 1.5666 - accuracy: 0.3438 - val_loss: 1.4973 - val_accuracy: 0.3253
Epoch 6/1000
12/12 [==============================] - 8s 695ms/step - loss: 1.5878 - accuracy: 0.3385 - val_loss: 1.8246 - val_accuracy: 0.3373
Epoch 7/1000
12/12 [==============================] - 7s 600ms/step - loss: 1.5779 - accuracy: 0.3333 - val_loss: 2.1682 - val_accuracy: 0.3494
Epoch 